# Data source: Global Energy Monitor

In [13]:
import pandas as pd

df_Units = pd.read_csv('../Units.csv')
df_Buses = pd.read_csv('../Buses.csv')
df_Units_modified = df_Units[(df_Units['type'] != 'interconnector') & (df_Units['type'] != 'demand_flex')]
df_Units_modified

,UnitID,name,type,Bus name,x,y,capacity,Technology,Cost
5,6,BFSE01,battery,way/90534270-400,-1.22,52.585,50.0,battery storage,0.00
6,7,AG-HLIM01,battery,way/263863442-400,-1.494,51.598,0.0,battery storage,0.00
7,8,AG-HLIM02,battery,way/4715504-400,-1.194,50.852,60.0,battery storage,0.00
8,9,AG-HLIM03,battery,way/95279334-400,-1.99,51.616,60.0,battery storage,0.00
9,10,AG-HLIM04,battery,way/263863442-400,-1.494,51.598,100.0,battery storage,0.00
...,...,...,...,...,...,...,...,...,...
1051,1052,GRGBW-2,offwind,way/576238223-400,1.713867,52.066,1075.0,offwind,0.02
1052,1053,GRGBW-3,offwind,way/576238223-400,1.713867,52.066,522.0,offwind,0.02
1053,1054,SGRWO-1,offwind,GB7-275,-1.741,56.588,36.9,offwind,0.02
1054,1055,ABRBO-1,offwind,GB7-275,-1.98333,57.2167,96.8,offwind,0.02


## Bioenergy

### Select datas from UK

In [14]:
df_bioenergy = pd.read_excel('Global-biomass.xlsx', sheet_name = 'Data')
df_bioenergy = df_bioenergy[df_bioenergy['Country/Area'] == 'United Kingdom']
# df_bioenergy = df_bioenergy[['Project Name', 'Longitude', 'Latitude', 'Capacity (MW)']]

df_Units_bioenergy = df_Units_modified[df_Units_modified['type'] == 'biomass']
df_bioenergy

,Date last researched,Country/Area,Project Name,Unit Name,Project Name in Local Language / Script,Other Name(s),Capacity (MW),Fuel,Status,Conversion/Replacement?,...,"Local Area (taluk, county)","Major Area (prefecture, district)",State/Province,Subregion,Region,GEM location ID,GEM phase ID,Other IDs (location),Other IDs (unit/phase),Wiki URL
0,2024-06-11,United Kingdom,3Rs Walsall power station,1,NaN,Walsall EFW,44.0,bioenergy: refuse (municipal and industrial wa...,pre-construction,NaN,...,NaN,West Midlands Combined Authority,England,Northern Europe,Europe,L100000201233,G100000201467,REPD: 12292,NaN,https://www.gem.wiki/3Rs_Walsall_power_station
27,2023-04-27,United Kingdom,Allington Quarry power station,1,NaN,"Allington Quarry Waste Management, Allington, ...",43.0,bioenergy: refuse (municipal and industrial wa...,operating,NaN,...,Kent,NaN,England,Northern Europe,Europe,L100000201234,G100000201468,REPD: 886,NaN,https://www.gem.wiki/Allington_Quarry_power_st...
120,2022-10-14,United Kingdom,Avonmouth ERF power station,1,NaN,Severn Road RRC,34.0,bioenergy: refuse (municipal and industrial wa...,operating,NaN,...,City of Bristol,West of England,England,Northern Europe,Europe,L100000200216,G100000200246,NaN,NaN,https://www.gem.wiki/Avonmouth_ERF_power_station
181,2023-04-26,United Kingdom,Blackburn Meadows power station,1,NaN,NaN,30.0,bioenergy: wood & other biomass (solids),operating,coal to bioenergy conversion,...,NaN,South Yorkshire Mayoral Combined Authority,England,Northern Europe,Europe,L100000200201,G100000200227,"World Resources Institute: Gbr0000166, REPD: 166",NaN,https://www.gem.wiki/Blackburn_Meadows_power_s...
199,2022-10-14,United Kingdom,Brigg Renewable Energy power station,1,NaN,NaN,40.0,"bioenergy: wood & other biomass (solids), bioe...",operating,NaN,...,North Lincolnshire,NaN,England,Northern Europe,Europe,L100000200224,G100000200254,REPD: 42,NaN,https://www.gem.wiki/Brigg_Renewable_Energy_po...
226,2022-10-14,United Kingdom,Cardiff ERF power station,1,NaN,Trident Park,30.0,bioenergy: refuse (municipal and industrial wa...,operating,NaN,...,Cardiff,NaN,Wales,Northern Europe,Europe,L100000200226,G100000200256,NaN,NaN,https://www.gem.wiki/Cardiff_ERF_power_station
347,2023-04-28,United Kingdom,Drax power station,"1, timepoint 2",NaN,UK Oxy CCS Demonstration Project (White Rose C...,645.0,bioenergy: wood & other biomass (solids),operating,coal to bioenergy conversion,...,Selby,North Yorkshire,England,Northern Europe,Europe,L100000103754,G1000002510,"REPD: 174, REPD: 192, REPD: 6659, REPD: 175, B...",Beyond Fossil Fuels: UK-66-1,https://www.gem.wiki/Drax_power_station
348,2023-04-28,United Kingdom,Drax power station,"2, timepoint 2",NaN,UK Oxy CCS Demonstration Project (White Rose C...,645.0,bioenergy: wood & other biomass (solids),operating,coal to bioenergy conversion,...,Selby,North Yorkshire,England,Northern Europe,Europe,L100000103754,G1000002511,"REPD: 174, REPD: 192, REPD: 6659, REPD: 175, B...",Beyond Fossil Fuels: UK-66-2,https://www.gem.wiki/Drax_power_station
349,2023-04-28,United Kingdom,Drax power station,"3, timepoint 2",NaN,UK Oxy CCS Demonstration Project (White Rose C...,645.0,bioenergy: wood & other biomass (solids),operating,coal to bioenergy conversion,...,Selby,North Yorkshire,England,Northern Europe,Europe,L100000103754,G1000002512,"REPD: 174, REPD: 192, REPD: 6659, REPD: 175, B...",Beyond Fossil Fuels: UK-66-3,https://www.gem.wiki/Drax_power_station
350,2023-04-28,United Kingdom,Drax power station,"4, timepoint 2",NaN,UK Oxy CCS Demonstration Project (White Rose C...,645.0,bioenergy: wood & other biomass (solids),operating,coal to bioenergy conversion,...,Selby,North Yorkshire,England,Northern Europe,Europe,L100000103754,G1000002513,"REPD: 174, REPD: 192, REPD: 6659, REPD: 175, B...",Beyond Fossil Fuels: UK-66-4,https://www.gem.wiki/Drax_power_station


### Map units based on lon and lat

In [15]:
len(df_bioenergy)

60

In [16]:
len(df_Units_modified[df_Units_modified['type'] == 'biomass'])

9

In [17]:
# import folium

# map_GB = folium.Map(location=[54, -2], zoom_start=6, tiles="CartoDB positron")
# for _, row in df_bioenergy.iterrows():

#     popup_html = f"""
#     <b>Power station:</b> {row.get("Project Name", "")}<br>
#     <b>x:</b> {row.get("Longitude", "")}<br>
#     <b>y:</b> {row.get("Latitude", "")}
#     """

#     folium.CircleMarker(
#             location=[row["Latitude"], row["Longitude"]],
#             radius=2,
#             color="yellow",
#             fill=True,
#             fill_opacity=0.7,
#             popup=folium.Popup(popup_html, max_width=200)
#         ).add_to(map_GB)


# for _, row in df_Units_bioenergy.iterrows():

#     popup_html = f"""
#     <b>UnitID:</b> {row.get("UnitID", "")}<br>
#     <b>x:</b> {row.get("x", "")}<br>
#     <b>y:</b> {row.get("y", "")}
#     """

#     folium.CircleMarker(
#             location=[row["y"], row["x"]],
#             radius=2,
#             color="purple",
#             fill=True,
#             fill_opacity=0.7,
#             popup=folium.Popup(popup_html, max_width=200)
#         ).add_to(map_GB)

# for _, row in df_Buses.iterrows():

#     popup_html = f"""
#     <b>UnitID:</b> {row.get("BusID", "")}<br>
#     <b>x:</b> {row.get("x", "")}<br>
#     <b>y:</b> {row.get("y", "")}
#     """

#     folium.CircleMarker(
#             location=[row["y"], row["x"]],
#             radius=2,
#             color="red",
#             fill=True,
#             fill_opacity=0.7,
#             popup=folium.Popup(popup_html, max_width=200)
#         ).add_to(map_GB)

# map_GB.save('Biomass_Visualization.html')

In [18]:
import numpy as np
from scipy.spatial import cKDTree

# Build KDTree from bioenergy points
tree = cKDTree(df_bioenergy[["Longitude", "Latitude"]].to_numpy())

# Query nearest bioenergy point for each unit
dist, idx = tree.query(df_Units_bioenergy[['x', 'y']].to_numpy(), k=1)

# Assign the nearest df_bioenergy index to each unit
df_Units_bioenergy = df_Units_bioenergy.copy()
df_Units_bioenergy["nearest_bioenergy_idx"] = idx
df_Units_bioenergy["nearest_bioenergy_lon"] = df_bioenergy.iloc[idx]["Longitude"].values
df_Units_bioenergy["nearest_bioenergy_lat"] = df_bioenergy.iloc[idx]["Latitude"].values
df_Units_bioenergy["nearest_bioenergy_capacity"] = df_bioenergy.iloc[idx]["Capacity (MW)"].values

In [19]:
df_Units_bioenergy['capacity'] = df_Units_bioenergy['nearest_bioenergy_capacity']
unit_counts = df_Units_bioenergy.groupby("nearest_bioenergy_idx")["UnitID"].transform('count')
df_Units_bioenergy["capacity"] = df_Units_bioenergy["nearest_bioenergy_capacity"] / unit_counts
df_Units_bioenergy = df_Units_bioenergy.loc[:, ['UnitID', 'name', 'type', 'Bus name', 'x', 'y', 'capacity', 'Technology', 'Cost', 'nearest_bioenergy_idx']]
df_Units_modified

,UnitID,name,type,Bus name,x,y,capacity,Technology,Cost
5,6,BFSE01,battery,way/90534270-400,-1.22,52.585,50.0,battery storage,0.00
6,7,AG-HLIM01,battery,way/263863442-400,-1.494,51.598,0.0,battery storage,0.00
7,8,AG-HLIM02,battery,way/4715504-400,-1.194,50.852,60.0,battery storage,0.00
8,9,AG-HLIM03,battery,way/95279334-400,-1.99,51.616,60.0,battery storage,0.00
9,10,AG-HLIM04,battery,way/263863442-400,-1.494,51.598,100.0,battery storage,0.00
...,...,...,...,...,...,...,...,...,...
1051,1052,GRGBW-2,offwind,way/576238223-400,1.713867,52.066,1075.0,offwind,0.02
1052,1053,GRGBW-3,offwind,way/576238223-400,1.713867,52.066,522.0,offwind,0.02
1053,1054,SGRWO-1,offwind,GB7-275,-1.741,56.588,36.9,offwind,0.02
1054,1055,ABRBO-1,offwind,GB7-275,-1.98333,57.2167,96.8,offwind,0.02


### After mapping to the nearest bioenergy power plant, for the other biomass plants, add them as new units and map them to the nearest buses.

In [20]:
new_units = []

for index, row in df_bioenergy.iterrows():
    if index in df_Units_bioenergy['nearest_bioenergy_idx']:
        continue
    
    new_unit = {
        'UnitID': f'new_bio_{index}',
        'name': f'Bioenergy_{index}',
        'type': 'biomass',
        # 'Bus name': None,
        'x': row['Longitude'],
        'y': row['Latitude'],
        'capacity': row['Capacity (MW)'],
        'Technology': 'biomass',
        'Cost': None,
    }
    new_units.append(new_unit)

    if new_units:
        df_new_units = pd.DataFrame(new_units)

        

### Match these units to nearest buses

In [21]:
bus_coords = df_Buses[['x', 'y']].to_numpy()
bus_ids = df_Buses['BusID'].values
bus_names = df_Buses['Bus name'].values
tree = cKDTree(bus_coords)

new_unit_coords = df_new_units[['x', 'y']].to_numpy()
dist, idx = tree.query(new_unit_coords, k=1)
nearest_bus_names = bus_names[idx]

df_new_units['Bus name'] = nearest_bus_names

df_Units_modified = pd.concat([df_Units_modified, df_new_units], ignore_index=True)

/var/folders/f0/zj71chhd6pvdlx6llbr36gtr0000gp/T/ipykernel_54100/2918477158.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_Units_modified = pd.concat([df_Units_modified, df_new_units], ignore_index=True)


In [22]:
df_Units_modified

,UnitID,name,type,Bus name,x,y,capacity,Technology,Cost
0,6,BFSE01,battery,way/90534270-400,-1.22,52.585,50.0,battery storage,0.0
1,7,AG-HLIM01,battery,way/263863442-400,-1.494,51.598,0.0,battery storage,0.0
2,8,AG-HLIM02,battery,way/4715504-400,-1.194,50.852,60.0,battery storage,0.0
3,9,AG-HLIM03,battery,way/95279334-400,-1.99,51.616,60.0,battery storage,0.0
4,10,AG-HLIM04,battery,way/263863442-400,-1.494,51.598,100.0,battery storage,0.0
...,...,...,...,...,...,...,...,...,...
482,new_bio_1682,Bioenergy_1682,biomass,GB73-400,0.3456,51.4645,45.0,biomass,NaN
483,new_bio_1784,Bioenergy_1784,biomass,way/91010022-400,0.7608,51.3659,50.0,biomass,NaN
484,new_bio_1789,Bioenergy_1789,biomass,way/97561542-275,-1.118,54.5895,30.0,biomass,NaN
485,new_bio_1790,Bioenergy_1790,biomass,way/97561542-275,-1.118,54.5895,49.0,biomass,NaN


## For different technologies

In [26]:
from assign_units import assign_tech_units

Technologies = ['biomass', 'hydro', 'nuclear']
for tech in Technologies:
    df_Units_modified = assign_tech_units(df_Units_modified=df_Units_modified, df_Buses=df_Buses, tech=tech)[1]
df_Units_modified


/Users/zm348/PhD/Projects/Nature-EV/data/Units_data_compare/assign_units.py:76: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_Units_modified = pd.concat([df_Units_modified, df_new_units], ignore_index=True)
/Users/zm348/PhD/Projects/Nature-EV/data/Units_data_compare/assign_units.py:76: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_Units_modified = pd.concat([df_Units_modified, df_new_units], ignore_index=True)
/Users/zm348/PhD/Projects/Nature-EV/data/Units_data_compare/assign_units.py:76: 

,UnitID,name,type,Bus name,x,y,capacity,Technology,Cost
0,6,BFSE01,battery,way/90534270-400,-1.22,52.585,50.0,battery storage,0.0
1,7,AG-HLIM01,battery,way/263863442-400,-1.494,51.598,0.0,battery storage,0.0
2,8,AG-HLIM02,battery,way/4715504-400,-1.194,50.852,60.0,battery storage,0.0
3,9,AG-HLIM03,battery,way/95279334-400,-1.99,51.616,60.0,battery storage,0.0
4,10,AG-HLIM04,battery,way/263863442-400,-1.494,51.598,100.0,battery storage,0.0
...,...,...,...,...,...,...,...,...,...
853,new_nuclear_1200,nuclear_1200,nuclear,way/49922626-400,-2.2649,50.6825,100.0,nuclear,NaN
854,new_nuclear_1201,nuclear_1201,nuclear,way/89427607-400,-4.4839,53.4113,1380.0,nuclear,NaN
855,new_nuclear_1202,nuclear_1202,nuclear,way/89427607-400,-4.4839,53.4113,1380.0,nuclear,NaN
856,new_nuclear_1203,nuclear_1203,nuclear,way/89427607-400,-4.4808,53.4161,530.0,nuclear,NaN
